In [20]:
import pandas as pd
import numpy as np
import altair as alt
import eco_style

alt.themes.enable('light')

ThemeRegistry.enable('light')

# Fig 1

In [220]:
df = pd.read_excel("fig1.xlsx", sheet_name="cleaned")
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y')

base = alt.Chart(df).encode(
    x=alt.X("management_score:Q", title="Management score"),
    y=alt.Y("density:Q", 
            title="Density",
            axis=alt.Axis(
                titleY=0,
                format=".1f")),
        
    color=alt.Color("date:N", title="Date", legend=alt.Legend(
        title=None,
        orient="top",
        legendY=-30
    )),
)

lines = base.mark_line(interpolate="basis")

chart = lines

chart.save("E60_fig1.json")
chart.save("E60_fig1.png", scale_factor=3.0)

chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

# Fig 2

In [225]:
df = pd.read_excel("fig2.xlsx", skiprows=4)
df = df[[c for c in df.columns if "Unnamed" not in str(c)]]
df = df.dropna()
df = df.melt(id_vars=["Section"], var_name="date", value_name="value")
df.columns = ["series", "date", "value"]
df

chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('value:Q', axis=alt.Axis(format=".0%")),
    y=alt.Y('date:N', title=None, axis=alt.Axis(labels=False)),
    color=alt.Color('date:N',scale=alt.Scale(
        range=["#0063AF", "#00A767"],
        reverse=True
    ),
    legend=alt.Legend(title=None, orient="top", legendY=-30)
    ),
    row=alt.Row('series:N', 
                header=alt.Header(labelAngle=0,
                                  labelAlign="left",
                                  ),
                title=None)
)

chart = chart.properties(
    height=alt.Step(20),
)

chart.save("E60_fig2.json")
chart.save("E60_fig2.png", scale_factor=3.0)
chart


/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

# Fig 3

In [222]:
df = pd.read_excel("fig3.xlsx", skiprows=5)
df = df.melt(id_vars="Density", var_name="series")
df['density'] = pd.to_numeric(df['Density'], errors='coerce')
df['management_score'] = pd.to_numeric(df['value'], errors='coerce')

df = df[['series', 'management_score', 'density']]
df = df.dropna()
dfs = []

chart

charts = []
for series in df['series'].unique().tolist()[:-1]:
    df_ = df.copy()
    colour_series = {
        series : 'Industry',
        'Whole economy': 'Whole economy',
    }
    df_['colour_series'] = df_['series'].map(colour_series, na_action='ignore')
    df_['colour_series'] = df_['colour_series'].fillna('Other Industries')
    colour_domain = ['Industry', 'Whole economy', 'Other Industries']
    colour_range = ['#E6224B', '#36B7B4', 'rgba(225, 225, 225, 1)']

    df_['order'] = df_['series'].apply(lambda x: 0 if x == series else 1 if x == 'Whole economy' else 2)
    df_ = df_.sort_values('order', ascending=False)

    dfs.append(df_)
    chart = alt.Chart(df_).encode(
        y=alt.Y('management_score:Q', title=None),
        x=alt.X('density:Q', title=None),
        color=alt.Color('colour_series:N', 
                        legend=alt.Legend(orient="top", title=None, legendY=-30),
                        title="Series", scale=alt.Scale(domain=colour_domain, range=colour_range)),
        size=alt.Size('order:Q', title=None, scale=alt.Scale(range=[3, 1.5]), legend=None),
        detail=alt.Detail('series:N')
    ).mark_line()

    chart = chart.properties(
        width=150,
        height=150,
        title=alt.TitleParams(series, anchor='start', orient='top', 
                                dx=20,
                                color="#676A86",
                                offset=0, 
                                fontSize=8.5 if series == 'Transport, storage, & communication' else 10)
    )
    charts.append(chart)

# shape the charts into 3 rows
rows = []
for i in range(0, len(charts), 3):
    row = charts[i:i+3]
    row = alt.hconcat(*row)
    rows.append(row)
chart = alt.vconcat(*rows)

chart.save("E60_fig3.json")
chart.save("E60_fig3.png", scale_factor=3.0)
chart



/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.VConcatChart(...)

In [197]:
dfs[3].query("order == 0")

,series,management_score,density,colour_series,order
446,"Transport, storage, & communication",0.111249,-0.03,Selected Industry,0
492,"Transport, storage, & communication",0.830946,0.43,Selected Industry,0
490,"Transport, storage, & communication",0.735522,0.41,Selected Industry,0
488,"Transport, storage, & communication",0.666263,0.39,Selected Industry,0
486,"Transport, storage, & communication",0.615319,0.37,Selected Industry,0
484,"Transport, storage, & communication",0.566962,0.35,Selected Industry,0
482,"Transport, storage, & communication",0.528229,0.33,Selected Industry,0
479,"Transport, storage, & communication",0.498791,0.30,Selected Industry,0
477,"Transport, storage, & communication",0.472264,0.28,Selected Industry,0
475,"Transport, storage, & communication",0.439867,0.26,Selected Industry,0


In [191]:
dfs[0].colour_series.value_counts()

colour_series
Other Industries     350
Selected Industry     50
Whole economy         50
Name: count, dtype: int64

In [172]:
dfs[0].query("colour_series == 'Selected Industry'")

for i in range(len(dfs)):
    print(dfs[i].query("colour_series == 'Selected Industry'").iloc[0].series)

Manufacturing
Construction
Construction
Transport, storage, & communication
Business services
Business services
Other services


In [157]:
all_series

['Non-manufacturing Production',
 'Manufacturing',
 'Construction',
 'Distribution, hotels & restaurants',
 'Transport, storage, & communication',
 'Business services',
 'Other services',
 'Real Estate',
 'Whole economy']

In [140]:
dfs[0].order.value_counts()

order
3    350
2     50
1     50
Name: count, dtype: int64

In [125]:
series

'Other services'

In [118]:
df_

,series,management_score,density,colour_series,order
0,Non-manufacturing Production,0.055726,-0.05,Other Industries,2
509,"Transport, storage, & communication",2.317487,0.60,Other Industries,2
525,"Transport, storage, & communication",1.854870,0.76,Other Industries,2
523,"Transport, storage, & communication",2.054385,0.74,Other Industries,2
521,"Transport, storage, & communication",2.270240,0.72,Other Industries,2
...,...,...,...,...,...
730,Other services,2.812917,0.59,Selected Industry,0
728,Other services,2.560572,0.57,Selected Industry,0
726,Other services,2.233146,0.55,Selected Industry,0
724,Other services,1.945775,0.53,Selected Industry,0


In [105]:
series

'Real Estate'

In [107]:
df

,series,management_score,density,colour_series,order
418,"Distribution, hotels & restaurants",0.994840,0.80,Other Industries,2
494,"Transport, storage, & communication",0.971767,0.45,Other Industries,2
352,"Distribution, hotels & restaurants",0.354060,0.14,Other Industries,2
356,"Distribution, hotels & restaurants",0.442054,0.18,Other Industries,2
367,"Distribution, hotels & restaurants",0.818213,0.29,Other Industries,2
...,...,...,...,...,...
784,Real Estate,0.176427,0.02,Selected Industry,0
878,Real Estate,0.023595,0.96,Selected Industry,0
861,Real Estate,1.912047,0.79,Selected Industry,0
803,Real Estate,0.180234,0.21,Selected Industry,0


In [94]:
df.series.unique()

array(['Distribution, hotels & restaurants', 'Construction',
       'Non-manufacturing Production', 'Manufacturing',
       'Business services', 'Transport, storage, & communication',
       'Real Estate', 'Other services', 'Whole economy'], dtype=object)

In [97]:
df.query("series == 'Real Estate'")

,series,management_score,density,colour_series,order
788,Real Estate,0.225741,0.06,Other Industries,2
821,Real Estate,0.919406,0.39,Other Industries,2
812,Real Estate,0.415054,0.30,Other Industries,2
786,Real Estate,0.209449,0.04,Other Industries,2
795,Real Estate,0.191696,0.13,Other Industries,2
850,Real Estate,3.001146,0.68,Other Industries,2
872,Real Estate,0.154202,0.90,Other Industries,2
874,Real Estate,0.094858,0.92,Other Industries,2
845,Real Estate,2.519359,0.63,Other Industries,2
841,Real Estate,1.798425,0.59,Other Industries,2
